## POS Tagging

Part-of-Speech (POS) tagging is a natural language processing technique that involves assigning specific grammatical categories or labels (such as nouns, verbs, adjectives, adverbs, pronouns, etc.) to individual words within a sentence.

In [1]:
import nltk
nltk.download('treebank')
from nltk.corpus import treebank
import random

# Load the tagged sentences from the treebank
tagged_sents = list(treebank.tagged_sents())

# Shuffle and split the data: 90% for training, 10% for testing
random.shuffle(tagged_sents)
split_index = int(0.9 * len(tagged_sents))
train_data = tagged_sents[:split_index]
test_data = tagged_sents[split_index:]

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


In [2]:
import math
from collections import defaultdict, Counter

class HMMPOSTagger:
    def __init__(self):
        # Count dictionaries to collect statistics from the training data
        self.initial_counts = Counter()
        self.transition_counts = defaultdict(Counter)
        self.emission_counts = defaultdict(Counter)
        self.tag_counts = Counter()
        self.vocab = set()
        self.tags = set()

    def train(self, tagged_sentences):
        """
        Trains the HMM POS tagger on a list of sentences.
        Each sentence is a list of (word, tag) pairs.
        """
        for sentence in tagged_sentences:
            if not sentence:
                continue
            # Update count for the first tag in each sentence
            first_word, first_tag = sentence[0]
            self.initial_counts[first_tag] += 1

            for i, (word, tag) in enumerate(sentence):
                self.tag_counts[tag] += 1
                self.emission_counts[tag][word] += 1
                self.vocab.add(word)
                self.tags.add(tag)
                # For transitions, consider the previous tag (if any)
                if i > 0:
                    prev_tag = sentence[i-1][1]
                    self.transition_counts[prev_tag][tag] += 1

        self.total_sentences = len(tagged_sentences)

        # Compute initial probabilities with add-one smoothing
        self.initial_probs = {}
        for tag in self.tags:
            self.initial_probs[tag] = math.log((self.initial_counts[tag] + 1) / (self.total_sentences + len(self.tags)))

        # Compute transition probabilities with add-one smoothing
        self.transition_probs = {}
        for prev_tag in self.tags:
            self.transition_probs[prev_tag] = {}
            total_transitions = sum(self.transition_counts[prev_tag].values())
            for curr_tag in self.tags:
                self.transition_probs[prev_tag][curr_tag] = math.log(
                    (self.transition_counts[prev_tag][curr_tag] + 1) /
                    (total_transitions + len(self.tags))
                )

        # Compute emission probabilities with add-one smoothing
        self.emission_probs = {}
        for tag in self.tags:
            self.emission_probs[tag] = {}
            total_emissions = sum(self.emission_counts[tag].values())
            for word in self.vocab:
                self.emission_probs[tag][word] = math.log(
                    (self.emission_counts[tag][word] + 1) /
                    (total_emissions + len(self.vocab))
                )
            # For words not seen during training, assign a small probability.
            self.emission_probs[tag]['<UNK>'] = math.log(1 / (total_emissions + len(self.vocab)))

    def viterbi(self, sentence):
        """
        Uses the Viterbi algorithm to decode the best tag sequence for a given sentence.
        sentence: list of words.
        Returns: list of predicted tags.
        """
        n = len(sentence)
        if n == 0:
            return []

        # delta holds the best scores for each tag at each word position.
        delta = [{} for _ in range(n)]
        # backpointer holds the best previous tag that led to the current tag.
        backpointer = [{} for _ in range(n)]

        # Initialization for the first word
        for tag in self.tags:
            word = sentence[0] if sentence[0] in self.vocab else '<UNK>'
            delta[0][tag] = self.initial_probs.get(tag, float('-inf')) + \
                            self.emission_probs[tag].get(word, self.emission_probs[tag]['<UNK>'])
            backpointer[0][tag] = None

        # Recursion through the sentence
        for t in range(1, n):
            for curr_tag in self.tags:
                word = sentence[t] if sentence[t] in self.vocab else '<UNK>'
                best_prev_tag = None
                best_score = float('-inf')
                for prev_tag in self.tags:
                    score = delta[t-1][prev_tag] + \
                            self.transition_probs[prev_tag].get(curr_tag, float('-inf')) + \
                            self.emission_probs[curr_tag].get(word, self.emission_probs[curr_tag]['<UNK>'])
                    if score > best_score:
                        best_score = score
                        best_prev_tag = prev_tag
                delta[t][curr_tag] = best_score
                backpointer[t][curr_tag] = best_prev_tag

        # Backtrack to find the best tag sequence
        best_final_tag = max(delta[n-1], key=delta[n-1].get)
        best_path = [best_final_tag]
        for t in range(n-1, 0, -1):
            best_path.insert(0, backpointer[t][best_path[0]])

        return best_path

In [3]:
tagger = HMMPOSTagger()
print("Training on {} sentences...".format(len(train_data)))
tagger.train(train_data)

# Evaluate on the test set
correct = 0
total = 0
for sentence in test_data:
    words = [word for word, tag in sentence]
    gold_tags = [tag for word, tag in sentence]
    predicted_tags = tagger.viterbi(words)
    total += len(gold_tags)
    correct += sum(1 for pt, gt in zip(predicted_tags, gold_tags) if pt == gt)

accuracy = correct / total
print("Accuracy on test set: {:.2%}".format(accuracy))

# Tag a sample sentence
sample_sentence = "The quick brown fox jumps over the lazy dog".split()
predicted_sample_tags = tagger.viterbi(sample_sentence)
print("Sample sentence:", sample_sentence)
print("Predicted tags:", predicted_sample_tags)

Training on 3522 sentences...
Accuracy on test set: 85.65%
Sample sentence: ['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']
Predicted tags: ['DT', 'JJ', 'NN', '.', "''", 'IN', 'DT', 'NN', 'IN']


In [4]:
# https://www.kaggle.com/code/nilaychauhan/part-of-speech-tagging-with-hidden-markov-models